## Gibbs Sampling with Blocks

In [1]:
import os
import time
import glob
# import warnings
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import itertools
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.special as scispe
import matplotlib.pyplot as plt
from scipy.linalg import toeplitz
# from numba import jit
# from numba.core.errors import (
#     NumbaWarning,
#     NumbaDeprecationWarning,
#     NumbaPendingDeprecationWarning
# )
from joblib import Parallel, delayed
# warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
# warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
# warnings.simplefilter("ignore", category=NumbaWarning)

### Draw from the Conditional Posterior of $(R^2, q)$

The conditional posterior of $(R^2, q)$ is given by:

\begin{equation*}
\pi(R^2, q | Y, U, X, \theta, z) \propto \\
\exp\left( -\dfrac{1}{2\sigma^2} \dfrac{k \bar{v}_x q (1-R^2)}{(R^2)} \beta^{\top} \text{diag}(z) \beta \right) \times \\
q^{s(z) + s(z)/2 + a - 1} (1 - q)^{k - s(z) + b - 1} \times (R^2)^{A - 1 - s(z)/2} (1 - R^2)^{s(z)/2 + B - 1},
\end{equation*}

where $s(z) = \sum_{i=1}^{k} z_i$. Discretize the support of $R^2$ and $q$ and sample from the discrete approximation.

In [13]:
# @jit(nopython=False)
def draw_r2_q(X, beta, sigma2, z, a, b, A, B, support_range_r2_q=None,
              return_posteriors=None, rng=None):
    """
    """
    k = len(beta)
    v_x = np.var(X, axis=0).mean()
    s_z = np.sum(z)

    # Define the ranges
    if support_range_r2_q:
        range_r2_q = support_range_r2_q
    else:
        EPSILON = 1e-5
        range_r2_q = np.concatenate([np.arange(0., 0.1, 0.001), 
                                     np.arange(0.11, 0.9, 0.01), 
                                     np.arange(0.901, 1.001, 0.001)])
        range_r2_q[0] += EPSILON
        range_r2_q[-1] -= EPSILON

    # Create a meshgrid
    R2, Q = np.meshgrid(range_r2_q, range_r2_q, indexing="ij", sparse=True)

    # Compute terms.
    beta_ = beta.reshape(beta.size, 1)
    expoterm = -0.5 / sigma2 * k * v_x * Q * (1 - R2)/R2 * (beta_.T * z @ beta_)
    pdf_q = Q ** (1.5 * s_z + a - 1) * (1 - Q) ** (k - s_z + b - 1)
    pdf_r2 = R2 ** (A - 0.5 * s_z - 1) * (1 - R2) ** (0.5 * s_z + B - 1)

    # Matrix of dimension `dim(support_q) x dim(support_r2)` 
    # with joint probabilities on (R2, q)
    posteriors = np.exp(expoterm) * pdf_r2 * pdf_q
    posteriors /= posteriors.sum()
    flat_posteriors = posteriors.flatten()

    if rng:
        index = rng.choice(flat_posteriors.size, p=flat_posteriors)
    else:
        index = np.random.choice(flat_posteriors.size, p=flat_posteriors)
    i, j = np.unravel_index(index, posteriors.shape)
    
    if return_posteriors:
        return (posteriors, None)  
    return range_r2_q[i], range_r2_q[j]

$$
R^2  = \dfrac{q k \gamma^2 \bar{v}_{x}}{q k \gamma^2 \bar{v}_{x} + 1} \Rightarrow \gamma^2 = \dfrac{R^2}{q k \bar{v}_{x} (1 - R^2)}
$$

In [14]:
def compute_gamma2(R2, q, k, v_x):
    return R2 / (q*k*v_x*(1-R2))

### Draw from the Conditional Posterior of $\phi$

The conditional posterior of $\phi$ is given by:

$$
\pi(\phi | Y, U, X, z, \beta, R^2, q, \sigma^2) = \pi(\phi | Y, U, X, \beta, \sigma^2) = \mathcal{N}((U'U)^{-1}U'(Y - X\beta), \sigma^2(U'U)^{-1}I).
$$



### Draw from the Conditional Posterior of $z$

The conditional posterior of $z$ is given by:

$$
\pi(z | Y, U, X, \phi, R^2, q) \propto q^{s(z)} (1 - q)^{k - s(z)} \left( \frac{1}{\gamma^2} \right)^{s(z) / 2} \left| W_e \right|^{-1/2} 
\left[ \dfrac{Y_e'Y_e - 2\beta_{eh}'W_e\beta_{eh}}{2} \right]^{-T/2} \Gamma\left( \frac{T}{2} \right),
$$

where 

* $\beta_{eh} := W_e^{-1}X_e'Y_e$, 

* $W_e := (X_e'X_e + I_{s(z)}/\gamma^2)$
  
* $Y_e := Y - U\phi$. 

Therefore, to sample from the posterior of $z|Y, U, X, ϕ, R2, q$  Draw iteratively from the distribution of $z_i | Y, U, X, \phi, R^2, q, z_{-i}$ using Gibbs sampling.



$$
\begin{align}
P_0 := P(z | z_i = 1) &\propto Q_{q, s(z), k, \gamma^2} \left| (W_1)_{s(z)} \right|^{-1/2} 
\left[ \dfrac{MSE_1}{2} \right]^{-T/2} \Gamma\left( \frac{T}{2} \right),
\\
P_1 := P(z | z_i = 0) &\propto Q_{q, (s(z) - 1), k, \gamma^2} \left| (W_0)_{s(z) - 1} \right|^{-1/2} 
\left[ \dfrac{MSE_0}{2} \right]^{-T/2} \Gamma\left( \frac{T}{2} \right),
\end{align}
$$

$$
\dfrac{P_1}{P_1 + P_0} = \dfrac{1}{1 + \frac{P_0}{P_1}} 
$$

In [15]:
# @jit(nopython=False)
def proba_z_i(Y, X, q, gamma_square, z, i):
    T = Y.shape[0]
    k = len(z)

    # Function to calculate W_e, beta_e_hat, det_W, and mse given z
    def calculate_statistics(z):
        s_z = np.sum(z)
        X_e = X[:, np.bool_(z)]
        I_sz = np.identity(s_z) if s_z > 0 else 1
        W_e = X_e.T @ X_e + I_sz / gamma_square
        beta_e_hat = np.linalg.inv(W_e) @ X_e.T @ Y
        det_W = np.linalg.det(W_e)
        mse = Y @ Y - beta_e_hat @ W_e @ beta_e_hat
        
        # print("s(z):", s_z)
        
        return W_e, beta_e_hat, det_W, mse

    # Pr(z_i=1)
    z[i] = 1
    W_e1, beta_e_hat1, det_W1, mse1 = calculate_statistics(z)

    # Pr(z_i=0)
    z[i] = 0
    W_e0, beta_e_hat0, det_W0, mse0 = calculate_statistics(z)
    
    z_proba = 1 / (1 + (det_W1 / det_W0)**0.5 * (mse1 / mse0)**T/2)
    
    return z_proba 

def draw_z_i(Y, X, q, gamma_square, z, i):
    zi_proba = proba_z_i(Y=Y, X=X, q=q, gamma_square=gamma_square, 
                         z=z, i=i)
    return 1 * (np.random.uniform() < zi_proba)

In [16]:
# @jit(nopython=False)
def gibbs_sampler_z(z_init, Y, X, q, gamma_square):
    z = z_init.copy()
            
    for i in range(z.shape[0]):
        # print(z)
        z[i] = draw_z_i(Y=Y, X=X, q=q, gamma_square=gamma_square, 
                        z=z, i=i)
    return z

### Draw from the Conditional Posterior of $\sigma^2$

The conditional posterior of $\sigma^2$ is given by:

$$
\pi(\sigma^2 | Y, U, X, \phi, R^2, q, z) = I_{\Gamma}\left( \frac{T}{2}, Y_e'Y_e - \beta_{\text{be}}'(X_e'X_e + I_{s(z)}/\gamma^2)\beta_{\text{be}} \right).
$$


In [17]:
# @jit(nopython=False)
def compute_active_components(Y, X, z, s_z, gamma_square):
    X_tilde = X[:, np.bool_(z)] if s_z > 0 else np.zeros(T).reshape(T, 1)
    I_sz = np.identity(s_z) if s_z > 0 else np.identity(1)
    W_tilde = X_tilde.T @ X_tilde + I_sz / gamma_square
    W_tilde_inv = np.linalg.inv(W_tilde)
    beta_tilde_hat =  W_tilde_inv @ X_tilde.T @ Y
    mse = Y @ Y - beta_tilde_hat @ W_tilde @ beta_tilde_hat

    return X_tilde, W_tilde_inv, mse



# @jit(nopython=False)
def draw_sigma2(T, mse):
    # Draw from the posterior
    sigma2 = stats.invgamma(T/2, scale=mse/2).rvs(size=1)[0]
    return sigma2

### Draw from the Conditional Posterior of $\beta_e := \tilde{\beta}$

The conditional posterior of $\beta_e$ is given by:

$$
\begin{align}
\pi(\beta_e | Y, U, X, \phi, R^2, q, \sigma^2, z) &= \mathcal{N}\left( (I_{s(z)}/\gamma^2 + X_e'X_e)^{-1}X_e'(Y - U\phi), \sigma^2 (I_{s(z)}/\gamma^2 + X_e'X_e)^{-1} \right) \\ 
&= \mathcal{N}\left( W_e^{-1}X_e'(Y - U\phi), \sigma^2 W_e^{-1} \right)
\end{align}
$$

In [18]:
def draw_beta_active(
    Y: np.ndarray, 
    X_tilde: np.ndarray, 
    W_tilde_inv: np.ndarray, 
    sigma2: float, 
    s_z: int, 
    verbose: bool=False
) -> np.ndarray:
    """Draw the active beta coefficients.

    Args:
        Y (np.ndarray): _description_
        X_tilde (np.ndarray): _description_
        W_tilde_inv (np.ndarray): _description_
        sigma2 (float): _description_
        s_z (int): _description_
        verbose (bool): _description_

    Returns:
        np.ndarray: _description_
    """
    mean = np.array(W_tilde_inv @ X_tilde.T @ Y)
    cov = np.array(sigma2 * W_tilde_inv)
    if verbose:
        print(f"X_e:{X_tilde[:1]}, mean:{mean[:1]}, cov:{cov[:1]}")
    # Draw from the posterior
    beta_active = np.random.multivariate_normal(mean, cov, size=s_z)[0]

    return beta_active

#### Generate the design matrix $X = (X_1,..., X_T), X_t = (X_{t1}, \dots, X_{tk}) \sim \mathcal{N}(\mu, \Sigma)$

$ \mu = \mathbb{0}_k, \Sigma = (\rho^{|i-j|})_{1 \leq i,j \leq k }$

In [19]:
def generate_X(T: int, k: int, rho: float) -> np.ndarray:
    """Generate a features matrix from a zero-mean Normal distribution with 
    a Toeplitz correlation with $corr(x_{t, i} - x_{t, j}) = \rho^{|i - j|}$.

    Args:
        T (int): number of observations.
        k (int): number of features.
        rho (float): base Toeplitz matrix correlation parameter.

    Returns:
        X (np.ndarray): features matrix.
    """
    # Create a matrix of difference i - j for all (i, j) in range(k) x range(k).
    diff = np.arange(k)[:, None] - np.arange(k)[None, :]

    # Compute a Toeplitz correlation matrix and define the mean vector.
    cov_matrix = rho ** np.abs(diff)
    mean = np.zeros(k)
    
    # Generate predictors X with Toeplitz correlation matrix.
    X = np.random.multivariate_normal(mean, cov=cov_matrix, size=T)
    X /= np.std(X, axis=0)

    return X

#### Draw $\tilde{\beta} = (\tilde{\beta}_1, \dots, \tilde{\beta}_s), \tilde{\beta}_i \overset{\tiny i.i.d.}{\sim} \mathcal{N}(0, 1) $

$S = (i_1,...,i_s) \sim RandomChoice(from=(1,...,k)), s \leq k$

$\beta_{j} = \tilde{\beta}_{j} \mathbb{I}{[j \in \{S\}]} $

In [20]:
def draw_regression_coefficients(k: int, s: int) -> np.ndarray:
    """
    Draw regression coefficients beta.

    Parameters:
        k (int): Total number of coefficients.
        s (int): Number of active coefficients.

    Returns:
        numpy.ndarray: Array of regression coefficients with 's' active indices.
    """
    beta = np.zeros(k)
    active_indices = np.random.choice(k, size=s, replace=False)
    beta[active_indices] = np.random.standard_normal(size=s)
    return beta

#### Simulate latent variable $z = (z_1,...,z_k), z_i \overset{\tiny i.i.d}{\sim} \mathcal{B}(s/k)$

In [21]:
def simulate_latent_variable(k: int, s: int) -> np.ndarray:
    """
    Simulate latent variable z.

    Parameters:
        k (int): Total number of latent variables.
        s (int): Number of active latent variables.

    Returns:
        numpy.ndarray: Array representing the simulated latent variable z.
    """
    z = np.random.binomial(1, p=s/k, size=k)
    return z

#### Draw error terms $\varepsilon \sim \mathcal{N}(0, \sigma^2 I_{T})$, $\sigma^2 = (1/Ry - 1) \sum_{t=1}^{T} (\beta'x_t)^2 / T$

In [22]:
def draw_error_terms(beta: np.ndarray, X: np.ndarray, T: int, Ry: float) -> np.ndarray:
    """
    Draw error terms epsilon.

    Parameters:
        beta (numpy.ndarray): Regression coefficients.
        X (numpy.ndarray): Design matrix.
        T (int): Number of observations.
        Ry (float): Variance of Y.

    Returns:
        numpy.ndarray: Array of error terms epsilon.
    """
    sigma2 = (1/Ry - 1) * np.sum((X @ beta)**2) / T
    cov_matrix = np.eye(T) * sigma2
    epsilon = np.random.multivariate_normal(mean=np.zeros(T), cov=cov_matrix, size=T)[0]
    return epsilon, sigma2

#### Compute response variable $Y = X \beta + \varepsilon$

In [23]:
def compute_response_variable(X: np.ndarray, beta: np.ndarray, epsilon: np.ndarray) -> np.ndarray:
    """
    Compute response variable Y.

    Parameters:
        X (numpy.ndarray): Design matrix of shape (T, k).
        beta (numpy.ndarray): Regression coefficients of shape (k,).
        epsilon (numpy.ndarray): Error terms of shape (T,).

    Returns:
        numpy.ndarray: Response variable Y of shape (T,).
    """
    Y = X @ beta + epsilon
    return Y

#### Simulate data. $(Y, X, \beta, z, \sigma^2) = f(s, R_y, T, k, \rho)$

In [29]:
def simulate_data(s: int, Ry: float, T: int = 200, k: int = 100, rho: float = 0.75
                  ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Simulate data for a regression model.

    Parameters:
        s (int): Number of active features.
        Ry (float): Percentage of explained variance.
        T (int): Number of observations (default is 200).
        k (int): Number of features (default is 100).
        rho (float): Autoregressive parameter for feature generation 
            (default is 0.75).

    Returns:
        Y (np.ndarray): Response variable.
        X (np.ndarray): Feature matrix.
        beta (np.ndarray): Regression coefficients.
        z (np.ndarray): Simulated latent variable (index of selected features).
    """
    # Generate features X.
    X = generate_X(T, k, rho)

    # Draw regression coefficients beta.
    beta = draw_regression_coefficients(k, s)

    # Simulate latent variable z.
    z = simulate_latent_variable(k, s)

    # Draw error terms epsilon.
    epsilon, sigma2 = draw_error_terms(beta, X, T, Ry)
    
    return beta, z, sigma2

In [36]:
import csv


def save_medians(q_medians, s, Ry, dataset_index, save_to="."):
    if "." not in save_to:
        os.makedirs(save_to, exist_ok=True)
        
    # Save q_medians to CSV
    with open(f"{save_to}/result_medians_s_{s}_Ry_{Ry}_dataset{dataset_index}.csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Key', 'Value'])
        csvwriter.writerows(q_medians.items())


def save_samples(q_sample, s, Ry, dataset_index, save_to="."):
    if "." not in save_to:
         os.makedirs(save_to, exist_ok=True)
    # Save q_sample to CSV
    with open(f"{save_to}/result_sample_s_{s}_Ry_{Ry}_dataset{dataset_index}.csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Key', 'Value'])
        csvwriter.writerows(q_sample.items())
        
def save_parameter_samples(parameter_samples, parameter_name, s, Ry, dataset_index, save_to="."):
    if "." not in save_to:
        os.makedirs(save_to, exist_ok=True)
    with open(f"{save_to}/result_{parameter_name}_s_{s}_Ry_{Ry}_dataset{dataset_index}.csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        for sample in parameter_samples:
            if isinstance(sample, np.ndarray):
                csvwriter.writerow(sample.tolist())
            elif isinstance(sample, (float, np.float64)):
                csvwriter.writerow([sample])
            else:
                csvwriter.writerow(sample)


def save_results2(q_medians_list, q_sample_list, save_to="."):
    if "." not in save_to:
        os.makedirs(save_to, exist_ok=True)
        
    with open(f"{save_to}/result_medians.csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Key', 'Value'])
        csvwriter.writerows(q_medians_list.items())
        
    with open(f"{save_to}/result_samples.csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Key', 'Value'])
        csvwriter.writerows(q_sample_list.items())

### Final Gibbs Sampling Procedure

In [34]:
def gibbs_sampling_parallel(
    X, 
    Y,
    s_range: List[int] = [5, 10, 100],
    Ry_range: List[float] = [0.02, 0.25, 0.50],
    T: int = 200,
    k: int = 100,
    rho: float = 0.75,
    a: float = 1,
    b: float = 1,
    A: float = 1,
    B: float = 1,
    n_datasets: int = 1,
    n_iter: int = 5_000,
    burn_in: int = 1_000,
    save_to: str = ".",
    n_jobs: int = -1
):

    q_medians_list = {} 
    q_sample_list = {}
    beta_list = {}
    sigma2_list = {}


    def process_pair(s, Ry):
        """
        Process a pair of (s, Ry) in parallel for multiple datasets.
        """
        print(f"(s, Ry) = {s, Ry}")
        start_time = time.time()

        local_q_medians = {}
        local_q_samples = {}
        local_beta_samples = {}  
        local_sigma2_samples = {} 

        for i in range(n_datasets):
            beta, z, sigma2 = simulate_data(s=s, Ry=Ry, T=T, k=k, rho=rho)
            v_x = np.var(X, axis=0).mean()
            q_list = []
            beta_samples = []  
            sigma2_samples = []  
            y_pred_samples = []

            for j in range(n_iter):
                R2, q = draw_r2_q(X, beta, sigma2, z, a, b, A, B)
                gamma2 = compute_gamma2(R2, q, k, v_x)
                z = gibbs_sampler_z(z, Y, X, q, gamma2)
                s_z = np.sum(z)

                X_tilde, W_tilde_inv, mse = compute_active_components(Y, X, z, s_z, gamma2)
                sigma2 = draw_sigma2(T, mse)

                if s_z == 0:
                    beta = np.zeros(beta.shape[0])
                else:
                    beta_tilde = draw_beta_active(Y, X_tilde, W_tilde_inv, sigma2, s_z)
                    beta = np.zeros(k)
                    beta[np.bool_(z)] = beta_tilde

                q_list.append(q)
                if j >= burn_in:
                    beta_samples.append(beta.copy())  
                    sigma2_samples.append(sigma2)  
                    y_pred = X @ beta
                    y_pred_samples.append(y_pred.reshape(-1))

            local_q_medians[f"{s}:{Ry}:{i}"] = np.median(q_list[burn_in:])
            local_q_samples[f"{s}:{Ry}:{i}:{j}"] = q_list[burn_in:]
            local_beta_samples[f"{s}:{Ry}:{i}"] = beta_samples
            local_sigma2_samples[f"{s}:{Ry}:{i}"] = sigma2_samples

            save_samples(local_q_samples, s, Ry, i, save_to=save_to)
            save_medians(local_q_medians, s, Ry, i, save_to=save_to)
            save_parameter_samples(local_beta_samples[f"{s}:{Ry}:{i}"], "beta", s, Ry, i, save_to=save_to)
            save_parameter_samples(local_sigma2_samples[f"{s}:{Ry}:{i}"], "sigma2", s, Ry, i, save_to=save_to)
            save_parameter_samples(y_pred_samples, "y_pred", s, Ry, i, save_to=save_to)


            print(f"Dataset no. {i}. Elapsed Time {time.time() - start_time}")

        return local_q_medians, local_q_samples, local_beta_samples, local_sigma2_samples


    results = Parallel(n_jobs=n_jobs)(
        delayed(process_pair)(s, Ry) for s, Ry in itertools.product(s_range, Ry_range)
    )

    for local_q_medians, local_q_samples, local_beta_list, local_sigma2_list in results:
        q_medians_list.update(local_q_medians)
        q_sample_list.update(local_q_samples)
        beta_list.update(local_beta_list)
        sigma2_list.update(local_sigma2_list)
    return q_medians_list, q_sample_list, beta_list, sigma2_list

In [38]:
def plot_beta_distribution(s, Ry):
    beta_file_path = f"./tests/result_beta_s_{s}_Ry_{Ry}_dataset0.csv"
    beta_data = pd.read_csv(beta_file_path, header=None)
    mean_betas = beta_data.mean()
    median_betas = beta_data.median()
    
    for index, (mean, median) in enumerate(zip(mean_betas, median_betas)):
        sns.histplot(beta_data[index], kde=True)
        plt.axvline(mean, color='red', linestyle='--', label='Mean')
        plt.axvline(median, color='green', linestyle='-', label='Median')
        plt.title(f'Beta coefficient distribution for {index}')
        plt.xlabel('Coefficient value')
        plt.ylabel('Frequency')
        plt.legend()
        plt.show()

def plot_q_distribution(s, Ry):
    q_samples_file_path = f"./tests/result_sample_s_{s}_Ry_{Ry}_dataset0.csv"
    q_samples_data = pd.read_csv(q_samples_file_path, header=None)
    q_samples = q_samples_data.iloc[:, 1].apply(lambda x: np.fromstring(x[1:-1], sep=','))

    for index in range(q_samples.shape[1]):
        sns.histplot(q_samples[index], kde=True)
        plt.title(f'Distribution of q values for parameter {index}')
        plt.xlabel('q value')
        plt.ylabel('Frequency')
        plt.show()

def plot_q_over_iterations(s, Ry):
    q_samples_file_path = f"./tests/result_sample_s_{s}_Ry_{Ry}_dataset0.csv"
    q_samples_data = pd.read_csv(q_samples_file_path, header=None)
    q_samples = q_samples_data.iloc[:, 1].apply(lambda x: np.fromstring(x[1:-1], sep=','))

    plt.figure(figsize=(12, 7))
    for index in range(q_samples.shape[1]):
        plt.plot(q_samples[index], label=f'Parameter {index}')
    plt.title('q Values over iterations')
    plt.xlabel('Iteration')
    plt.ylabel('q Value')
    plt.legend()
    plt.show()

In [3]:
def compute_median_q_values(q_samples_file_path):
    q_samples_data = pd.read_csv(q_samples_file_path, header=None)
    q_values = q_samples_data.iloc[:, 1].apply(lambda x: np.fromstring(x[1:-1], sep=','))
    q_values_df = pd.DataFrame(q_values.tolist())
    median_qs = q_values_df.median()
    return median_qs

def filter_beta_coefficients_mean(beta_file_path, median_qs, q_threshold):
    betas_data = pd.read_csv(beta_file_path, header=None)
    filtered_betas = betas_data.where(median_qs >= q_threshold, 0)
    mean_filtered_betas = filtered_betas.mean()
    return mean_filtered_betas